In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Load the weather data and merge them into our two DataFrames.'''
path = r'D:\Springboard_DataSci\Assignments\Capstone_2--Airport_weather\data'
os.chdir(path)
print('Loading weather events')
weather_events = pd.read_csv('weather_events.csv')
print('Loading flight data')
departing_flights = pd.read_csv('departing_flights.csv')
arriving_flights = pd.read_csv('arriving_flights.csv')

Loading weather events
Loading flight data


In [3]:
# Merge the DataFrames.
departure_events = departing_flights.merge(
    weather_events, how='left', left_on=['ORIGIN','DepartureDate'],
    right_on=['Airport','Date'], validate='one_to_many')
arrival_events = arriving_flights.merge(
    weather_events, how='left', left_on=['DEST','ArrivalDate'],
    right_on=['Airport','Date'], validate='one_to_many')

In [4]:
# Drop redundant columns.
departure_events.drop(['Airport','Date'], axis=1, inplace=True)
arrival_events.drop(['Airport','Date'], axis=1, inplace=True)

# Clean up the column names.
inputVars = pd.Series(['Cold','Fog','Hail','Rain','Snow','Storm'], name='Code')
column_renames = {'ARR_DEL15':'ArrivDel',
                  'DEP_DEL15':'DepartDel',
                  'ColdCode':'Cold',
                  'FogCode':'Fog',
                  'HailCode':'Hail',
                  'RainCode':'Rain',
                  'SnowCode':'Snow',
                  'StormCode':'Storm'}
departure_events.rename(columns=column_renames, inplace=True)
arrival_events.rename(columns=column_renames, inplace=True)

# NaN's mean no weather events on that day. Fill them in with 0's.
for column in inputVars:
    departure_events[column].fillna(0, inplace=True)
    arrival_events[column].fillna(0, inplace=True)

In [6]:
'''Exploratory data analysis'''
FRAC_CANCELLED = 'FracCancelled'
FRAC_DELAYED = 'FracDelayed'
departure_events[FRAC_CANCELLED] = departure_events.WeatherCancelled/departure_events.Flights
arrival_events[FRAC_CANCELLED] = arrival_events.WeatherCancelled/arrival_events.Flights
departure_events[FRAC_DELAYED] = departure_events.WeatherDelayed/departure_events.Flights
arrival_events[FRAC_DELAYED] = arrival_events.WeatherDelayed/arrival_events.Flights

X_dep = departure_events.loc[:, inputVars]
X_arr = arrival_events.loc[:, inputVars]

In [7]:
#Get the correlation matrix of data with X's features as columns.
def corrMatrixAndMax(df, labels=inputVars):
    corr = df.corr()
    n = corr.shape[0]
    max_corr_ID = np.argmax(np.abs(corr) - np.eye(n))
    return corr, np.unravel_index(max_corr_ID, [n,n])

#Get the value, and row and column labels of a particular df coordinate. Rounding is arbitrary.
def getCorrNameCoords(df, coordinates):
    row = coordinates[0]; col = coordinates[1]
    return round(df.iloc[row, col], 3), df.index[row], df.index[col]
        
#Max absolute difference.
maxAbsDiff = lambda A, B, axis=None: np.max(np.abs(A-B), axis=axis)

In [8]:
print('Deviation from X_dep and array of ints:', maxAbsDiff(X_dep, X_dep.astype(int)) )
print('\nDeviation from X_arr and array of ints:', maxAbsDiff(X_arr, X_arr.astype(int)) )
X_dep = X_dep.astype(int)
X_arr = X_arr.astype(int)

print('\nCalculating correlation matrices')
Corr_X_dep, coords_max_Corr_X_dep = corrMatrixAndMax(X_dep)
Corr_X_arr, coords_max_Corr_X_arr = corrMatrixAndMax(X_arr)
print('Max abs difference between Corr_X_dep and Corr_X_arr:', maxAbsDiff(Corr_X_dep, Corr_X_arr))
#Nearly identical, which makes sense given that they're from the same dates and airports.

Deviation from X_dep and array of ints: Cold     0.0
Fog      0.0
Hail     0.0
Rain     0.0
Snow     0.0
Storm    0.0
dtype: float64

Deviation from X_arr and array of ints: Cold     0.0
Fog      0.0
Hail     0.0
Rain     0.0
Snow     0.0
Storm    0.0
dtype: float64

Calculating correlation matrices
Max abs difference between Corr_X_dep and Corr_X_arr: Cold     0.000011
Fog      0.000403
Hail     0.000040
Rain     0.000403
Snow     0.000040
Storm    0.000054
dtype: float64


In [9]:
EV_CORR_MTRX = ' event correlation matrix:\n'
CORR_NAME_COORDS = 'Greatest correlation and name and coordinates:'
print('\nDeparture' + EV_CORR_MTRX + str(Corr_X_dep))
print(CORR_NAME_COORDS, getCorrNameCoords(Corr_X_dep, coords_max_Corr_X_dep))
print('\nArrival' + EV_CORR_MTRX + str(Corr_X_arr))
print(CORR_NAME_COORDS, getCorrNameCoords(Corr_X_arr, coords_max_Corr_X_arr))


Departure event correlation matrix:
           Cold       Fog      Hail      Rain      Snow     Storm
Cold   1.000000  0.008870  0.014899 -0.004855  0.024090  0.022348
Fog    0.008870  1.000000  0.017653  0.111326  0.163653  0.029486
Hail   0.014899  0.017653  1.000000  0.068580  0.178120  0.016327
Rain  -0.004855  0.111326  0.068580  1.000000  0.008625  0.091570
Snow   0.024090  0.163653  0.178120  0.008625  1.000000  0.035830
Storm  0.022348  0.029486  0.016327  0.091570  0.035830  1.000000
Greatest correlation and name and coordinates: (0.178, 'Hail', 'Snow')

Arrival event correlation matrix:
           Cold       Fog      Hail      Rain      Snow     Storm
Cold   1.000000  0.008874  0.014903 -0.004844  0.024100  0.022352
Fog    0.008874  1.000000  0.017657  0.111729  0.163612  0.029488
Hail   0.014903  0.017657  1.000000  0.068540  0.178129  0.016333
Rain  -0.004844  0.111729  0.068540  1.000000  0.008646  0.091516
Snow   0.024100  0.163612  0.178129  0.008646  1.000000  0.035847

This is suspicious. Hail is a warm-weather event; snow is a cold-weather event. 
Let's look at the Hail and Snow codes to see how much overlap there is.

In [10]:
def SumByPair(df, label1, label2):
    return df.groupby([label1, label2]).size().reset_index().rename(columns={0:'count'})
print('\nDepartures:\n' + str(SumByPair(X_dep, 'Hail', 'Snow')))
print('\nArrivals:\n' + str(SumByPair(X_arr, 'Hail', 'Snow')))


Departures:
   Hail  Snow  count
0     0     0  22054
1     0     1    484
2     0     2    173
3     0     3     97
4     1     0     37
5     1     1     10
6     1     2     11
7     1     3     14

Arrivals:
   Hail  Snow  count
0     0     0  22086
1     0     1    484
2     0     2    173
3     0     3     97
4     1     0     37
5     1     1     10
6     1     2     11
7     1     3     14


Nearly half the Hail=1 codes are correlated with Snow>0 codes. This is a problem,
and we need to investigate it. Let's take a look at these 35 days in the departure
and arrival events DataFrames.

In [12]:
hailAndSnowDep = departure_events[
    (departure_events.Hail==1) & (departure_events.Snow>0)]
hailAndSnowArr = arrival_events[
    (arrival_events.Hail==1) & (arrival_events.Snow>0)]

# The date/airport overlap is probably large. Let's compare these two DFs.
hailAndSnowEvents = hailAndSnowDep[['ORIGIN','DepartureDate','Hail','Snow']].merge(
    hailAndSnowArr[['DEST','ArrivalDate','Hail','Snow']], how='outer',
    left_on=['ORIGIN','DepartureDate'], right_on=['DEST','ArrivalDate'],
    suffixes=('_dep','_arr'))

clashes = hailAndSnowEvents.apply(
    lambda row: any((row['ORIGIN']!=row['DEST'], row['Hail_dep']!=row['Hail_arr'],
                     row['DepartureDate']!=row['ArrivalDate'], row['Snow_dep']!=row['Snow_arr'])), axis=1)
print('Any clashes between departure/arrival columns?', clashes.any())
# None. We can just use hailAndSnowDep or hailAndSnowArr.

In [ ]:
hailAndSnowDep = hailAndSnowDep[['ORIGIN','DepartureDate','Hail','Snow']]
del(hailAndSnowArr, hailAndSnowEvents)

In [ ]:
print('\nOverlapping hail and snow days:')
print('\n' + str(hailAndSnowDep))

In [ ]:
nonzero_X_dep = X_dep[(X_dep.T != 0).any()]
nonzero_X_arr = X_arr[(X_arr.T != 0).any()]
nonzero_Corr_X_dep, nonzero_coords_max_Corr_X_dep = corrMatrixAndMax(nonzero_X_dep)
nonzero_Corr_X_arr, nonzero_coords_max_Corr_X_arr = corrMatrixAndMax(nonzero_X_arr)
print('Max abs difference between nonzero_Corr_X_dep and nonzero_Corr_X_arr:',
      maxAbsDiff(nonzero_Corr_X_dep, nonzero_Corr_X_arr))
#Identical correlation matrices, so we don't have to separate arrival/departure here.

print('\nNonzero' + EV_CORR_MTRX, nonzero_Corr_X_dep)
print(CORR_NAME_COORDS, getCorrNameCoords(nonzero_Corr_X_dep, nonzero_coords_max_Corr_X_dep))

Now the greatest correlation is between rain and fog, at about -0.297. Paradoxically,
this correlation was positive with all zero rows included! Perhaps we have the same
effect here as with hail and cold. Let's find out.

In [ ]:
print('\nDepartures:\n' + str(SumByPair(X_dep, 'Rain', 'Fog')))
print('\nDepartures (fog first):\n' + str(SumByPair(X_dep, 'Fog', 'Rain')))

In [ ]:
print('\nNonzero rows:\n' + str(SumByPair(nonzero_X_dep, 'Rain', 'Fog')))
print('\nNonzero rows (fog first):\n' + str(SumByPair(nonzero_X_dep, 'Fog', 'Rain')))

Nearly all the 0-0 rows got cut. All other rows are preserved. The negative correlation after
the cut shows that an overlap between rain and fog is uncommon. Intuitively this makes sense, as
fog is generally a fair-weather event and rain is generally a poor-weather event. Incidentally,
note that light rain is more likely when the fog is heavy.

Because the untrimmed departure and arrival correlation matrices are nearly identical,
we will just use the departure events now. Let's see what the average values per
column are, weighted by flight.

In [ ]:
Avg = pd.DataFrame(map(list, zip(inputVars.values, np.average(
    X_dep, axis=0, weights=departure_events.Flights))), columns=['Code', 'WtAvg'])
Avg = Avg.set_index('Code')['WtAvg']
print('\nWeighted averages: ' + str(Avg) + '\n')

Now let's do some plots, per code, categorized by code value. To make things
simpler we will not weight these values for now. There are a ton of outliers; we
will do the plots both with and without them.

In [ ]:
frac_cancelled = departure_events['FracCancelled']
frac_delayed = departure_events['FracDelayed']

for showfliers in [True, False]: #Outliers
    for weatherCode in inputVars:
        unique_codes = np.sort(X_dep[weatherCode].unique()) #makes a Series of value counts indexed by code #
        cancelled_per_code = [frac_cancelled[X_dep[weatherCode]==j] for j in unique_codes]
        delayed_per_code = [frac_delayed[X_dep[weatherCode]==j] for j in unique_codes]
    
        fig, axs = plt.subplots(1,2)
        axs[0].boxplot(cancelled_per_code, showfliers=showfliers)
        axs[0].set_title('Cancellation fraction per code')
        axs[1].boxplot(delayed_per_code, showfliers=showfliers)
        axs[1].set_title('Delay fraction per code')
        for j in [0,1]:
            axs[j].set_xticklabels(unique_codes)
            axs[j].set_xlabel('Value of "' + weatherCode + '"')
            axs[j].yaxis.grid(True)
        if not showfliers:
            plt.savefig('BoxPlot' + weatherCode + '.png')
        plt.tight_layout()
        plt.show()


The medians of some of these features clearly are affected by the feature value;
however, none of them seem to add up to much more than about 0.20, meaning that
we should not expect that these weather events will have an overwhelming affect
on delays and cancellations. Still, there do seem to be trends that emerge.

In [5]:
# Save the results.
departure_events.to_csv('departure_events.csv') #Uncomment to save.
arrival_events.to_csv('arrival_events.csv')